In [1]:
import openai

In [36]:
get_data_func = {
    "name": "get_data",
    "description": "Fetches review data to be analysed. This can either be in the form of raw reviews or statistics calculated by a sql query. For complex questions, divide them into sub-questions and create a function call for each sub-question.",
    "parameters": {
        "type": "array",
        "properties": {
                "source": {
                    "type": "string",
                    "description": "Data source. Set as 'app' for mobile bank app reviews and 'bank' for general bank reviews.",
                },
                "start_date": {
                    "type": "string",
                    "description": "Start date of the date range. Format: YYYY-MM-DD. Current data: 2023-08-15",
                },
                "end_date": {
                    "type": "string",
                    "description": "End date of the date range. Format: YYYY-MM-DD",
                },
                "category": {
                    "type": "string",
                    "description": "Type of review",
                    "enum": [
                        "other",
                        "functionality",
                        "ui",
                        "error",
                        "stability",
                        "customer service",
                        "counseling",
                        "mobile/web bank",
                        "fees and interest rates",
                    ],
                },
                "bank": {"type": "string", "description": "Bank name"},
                "country": {"type": "string", "description": "Country name"},
                "similarity_query": {
                    "type": "string",
                    "description": "Semantic similarity query text. Use to get reviews with similar meaning to the similarity_query.",
                },
                "retrieval_method": {
                    "type": "string",
                    "description": "Set as 'reviews' to retrieve reviews and 'statistics' to retrieve statistics. If not specified, the default value is 'reviews'.",
                },
        },
        "required": ["source"],
    },
}

In [37]:
prompt = "Hvordan har kunderne anmeldt Danske Bank appen i forhold til SDC appen?"

res = openai.ChatCompletion.create(
    model='gpt-4-0613',  # swap for gpt-3.5-turbo-0613 if needed
    messages=[{"role": "user", "content": prompt}],
    functions=[get_data_func]
)

InvalidRequestError: Invalid schema for function 'get_data': schema must be a JSON Schema of 'type: "object"', got 'type: "array"'.

In [35]:
res['choices'][0]

<OpenAIObject at 0x12abc96d0> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "get_data",
      "arguments": "{\n  \"source\": \"app\",\n  \"bank\": \"Danske Bank\",\n  \"retrieval_method\": \"statistics\"\n}"
    }
  },
  "finish_reason": "function_call"
}

In [27]:
print(res['choices'][0].message["function_call"]["arguments"])

{
  "source": "app",
  "bank": "Danske Bank",
  "retrieval_method": "statistics"
}


In [22]:
res['choices'][0]

<OpenAIObject at 0x12ab17e90> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "get_data",
      "arguments": "{\n  \"source\": \"app\",\n  \"bank\": \"Danske Bank\",\n  \"retrieval_method\": \"statistics\"\n}"
    }
  },
  "finish_reason": "function_call"
}

In [ ]:
"""
1. source: Mandatory field. Set as 'app' for mobile bank app reviews and 'bank' for general bank reviews.
2. date_range: Include 'start' and 'end' fields in 'yyyy-mm-dd' format when specified in the question, otherwise, omit this field. The current date is {{ current_date }}.
3. category: Set this field to one of the given values when a review type is mentioned, otherwise, omit it: 'other', 'functionality', 'ui', 'error', 'stability', 'customer service', 'counseling', 'mobile/web bank', 'fees and interest rates'.
4. bank: Mention the bank's name if the question is about a specific bank, otherwise, omit this field.
5. country: Use the respective country code (e.g., 'DK', 'NO', 'SE', 'FO', 'UK', 'NZ') if a specific country is mentioned, otherwise, omit this field.
6. similarity_query: Set this field with the query text if a specific semantic query is mentioned, otherwise, omit it.
7. retrieve: Set as 'reviews' to retrieve reviews and 'statistics' to retrieve statistics. If not specified, the default value is 'reviews'.

For complex questions, divide them into sub-questions and refine results using the parameters 'date_range', 'category', 'bank', and 'country', with the optional 'similarity_query' for querying or ordering by semantic similarity. Ensure you accurately transform the question into the JSON structure, taking care to attend to the subtleties and details provided in the prompt."""

"""

In [ ]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]


response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
       {"role": "user", "content": "Explain how to assemble a PC"}
    ],
    functions=[
        {
          "name": "get_answer_for_user_query",
          "description": "Get user answer in series of steps",
          "parameters": StepByStepAIResponse.schema()
        }
    ],
    function_call={"name": "get_answer_for_user_query"}
)